In [5]:
# README: The scope of this project is limited to Pokemon from generations 1-5 (games that were playable on the Nintendo DS lite)

### *This notebook serves as the master Pokemon data notebook. Other notebooks that are more narrowly focused on specific Pokemon data will run this notebook and use its functionality to simplify code writing and streamline functions like joining and webscraping.

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from IPython.display import HTML
# Maximize display of all dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('future.no_silent_downcasting', True)

### Web Scraping Function

In [9]:
# Scrape data from a web server
def scrape_table(url):
    response = requests.get(url)
    if response.status_code != 200:  
        return None
    soup = BeautifulSoup(response.text, 'html.parser')
    rows = soup.find_all('tr') # Retrieve all rows within the database
    data = []
    for row in rows:
        cols = row.find_all(['td','th']) # Retrieve all columns within each row
        row_data = []
        # Account for columns that may contain an images as values
        for col in cols:
            img = col.find('img')  # Check for an image inside each cell
            if img and not col.text.strip():
                img_url = f'https://www.serebii.net{img['src']}' # Get full image url 
                row_data.append(img_url) 
            else:
                row_data.append(col.text.strip())  # Otherwise, get text
        data.append(row_data)
    global df
    df = pd.DataFrame(data)
    return df

### Join Function

In [11]:
# Function to add new columns to an existing dataframe 
def join(left_df, right_df):
    left_df = pd.merge(left_df, right_df, how = 'left', on = 'Pokémon')
    return left_df

### Pass National Pokedex URL into Scraping Function and Prepare Dataframe for Cleaning

In [13]:
pokedex_url = 'https://pokemondb.net/pokedex/all'
scrape_table(pokedex_url)
pokedex_df = df
# Realign dataframe
pokedex_df.columns = pokedex_df.iloc[0] 
pokedex_df = pokedex_df.iloc[1:]   
pokedex_df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
1,0001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
2,0002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
3,0003,Venusaur,Grass Poison,525,80,82,83,100,100,80
4,0003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
5,0004,Charmander,Fire,309,39,52,43,60,50,65


#### Apply Transformations to pokedex_df

In [15]:
pokedex_df = pokedex_df.rename(columns = {'Total':'Base Stats', 'Name':'Pokémon'})
# Clean up alternate form names
pokedex_df['Pokémon'] = pokedex_df['Pokémon'].str.upper().str.strip().replace({'ROTOM HEAT ROTOM':'ROTOM HEAT FORME','ROTOM WASH ROTOM':'ROTOM WASH FORME','ROTOM FROST ROTOM':'ROTOM FROST FORME','ROTOM FAN ROTOM':'ROTOM FAN FORME','ROTOM MOW ROTOM':'ROTOM MOW FORME','GIRATINA ALTERED FORME':'GIRATINA','GIRATINA ORIGIN FORME':'GIRATINA DISTORTION FORME','BASCULIN RED-STRIPED FORM':'BASCULIN','DARMANITAN STANDARD MODE':'DARMANITAN','KYUREM WHITE KYUREM':'KYUREM WHITE','KYUREM BLACK KYUREM':'KYUREM BLACK','TORNADUS INCARNATE FORME':'TORNADUS','THUNDURUS INCARNATE FORME':'THUNDURUS','LANDORUS INCARNATE FORME':'LANDORUS','MELOETTA ARIA FORME':'MELOETTA','KELDEO ORDINARY FORM':'KELDEO'})
pokedex_df = pokedex_df[~pokedex_df['Pokémon'].isin(['BASCULIN BLUE-STRIPED FORM','BASCULIN WHITE-STRIPED FORM'])]
# Remove alternative post-gen5 forms (e.g. Mega Charizard, Dialga Origin Forme) 
pokedex_df = pokedex_df[~pokedex_df['Pokémon'].str.contains('MEGA|GALARIAN|HISUIAN|ALOLAN|PALDEAN|PRIMAL|PARTNER|BREED|ORIGIN', regex=True, case=False)]
# Remove "fairy" type designation and replace with "normal" type, as this dataframe is mean to include only generation 1-5 data
pokedex_df['Type'] = pokedex_df['Type'].str.replace(' ','|').str.replace(r'(^Fairy\||\|?Fairy)', '', regex=True).replace('','Normal')
pokedex_df.loc[pokedex_df['Pokémon'].isin(['TOGETIC', 'TOGEKISS']), 'Type'] = 'Normal|Flying'
pokedex_df['#'] = pokedex_df['#'].astype('int')
pokedex_df = pokedex_df[pokedex_df['#'] <= 649]  
# Assign generation based on pokedex #
def assign_gen(input):
    if input <= 151:
        return '1'
    if input > 151 and input <= 251:
        return '2'
    if input > 251 and input <= 386:
        return '3'
    if input > 386 and input <= 493:
        return '4'
    else:
        return '5'
pokedex_df['Generation'] = pokedex_df['#'].apply(assign_gen)
# Assign a Boolean to Pokemon that exist in the regional Kanto pokedex from the video game. Pokemon from a single generation may also exist in more than one regional pokedex. 
pokedex_df['Kanto Pokedex'] = np.where(pokedex_df['#'] <= 151, True, False)
pokedex_df.head()

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex
1,1,BULBASAUR,Grass|Poison,318,45,49,49,65,65,45,1,True
2,2,IVYSAUR,Grass|Poison,405,60,62,63,80,80,60,1,True
3,3,VENUSAUR,Grass|Poison,525,80,82,83,100,100,80,1,True
5,4,CHARMANDER,Fire,309,39,52,43,60,50,65,1,True
6,5,CHARMELEON,Fire,405,58,64,58,80,65,80,1,True


### Pass Johto Pokedex URL into Scraping Function and Prepare Dataframe for Join

In [17]:
# This dataset contains Pokemon that are registered in the Johto regional pokedex (Pokemon Gold/Silver/HeartGold/Soulsilver), a combination of generation 1 and generation 2 Pokemon
johto_url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_Johto_Pok%C3%A9dex_number'
scrape_table(johto_url)
johto_pokedex = df
# Apply transformations
johto_pokedex.columns = johto_pokedex.iloc[1]
johto_pokedex = johto_pokedex.iloc[2:]
johto_pokedex = johto_pokedex[['Pokémon']]
johto_pokedex['Pokémon'] = johto_pokedex['Pokémon'].str.upper()
johto_pokedex = johto_pokedex[johto_pokedex['Pokémon'] != 'Pokémon']
johto_pokedex = johto_pokedex[johto_pokedex.index <= 261]
johto_pokedex['Johto Pokedex'] = True
johto_pokedex = johto_pokedex.drop_duplicates()
johto_pokedex.head()

1,Pokémon,Johto Pokedex
2,CHIKORITA,True
3,BAYLEEF,True
4,MEGANIUM,True
5,CYNDAQUIL,True
6,QUILAVA,True


#### Merge Gen 2 Dataframe to Pokedex

In [19]:
pokedex_df = join(pokedex_df, johto_pokedex)
pokedex_df.head()

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex
0,1,BULBASAUR,Grass|Poison,318,45,49,49,65,65,45,1,True,True
1,2,IVYSAUR,Grass|Poison,405,60,62,63,80,80,60,1,True,True
2,3,VENUSAUR,Grass|Poison,525,80,82,83,100,100,80,1,True,True
3,4,CHARMANDER,Fire,309,39,52,43,60,50,65,1,True,True
4,5,CHARMELEON,Fire,405,58,64,58,80,65,80,1,True,True


### Pass Hoenn Pokedex URL into Scraping Function and Prepare Dataframe for Join

In [21]:
# This dataset contains Pokemon that are registered in the Hoenn regional pokedex (Pokemon Ruby/Sapphire/Emerald), a combination of generations 1-3 Pokemon
hoenn_url='https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_Hoenn_Pok%C3%A9dex_number_in_Generation_III'
scrape_table(hoenn_url)
hoenn_pokedex = df
# Apply transformations
hoenn_pokedex.columns = hoenn_pokedex.iloc[1]
hoenn_pokedex = hoenn_pokedex.iloc[2:]
hoenn_pokedex = hoenn_pokedex[['Pokémon']]
hoenn_pokedex['Pokémon'] = hoenn_pokedex['Pokémon'].str.upper()
hoenn_pokedex = hoenn_pokedex[hoenn_pokedex['Pokémon'] != 'Pokémon']
hoenn_pokedex = hoenn_pokedex[hoenn_pokedex.index <= 209]
hoenn_pokedex['Hoenn Pokedex'] = True
hoenn_pokedex = hoenn_pokedex.drop_duplicates()
hoenn_pokedex.head()

1,Pokémon,Hoenn Pokedex
2,TREECKO,True
3,GROVYLE,True
4,SCEPTILE,True
5,TORCHIC,True
6,COMBUSKEN,True


#### Merge Gen 3 Dataframe to Pokedex

In [23]:
pokedex_df = join(pokedex_df, hoenn_pokedex)
pokedex_df.loc[pokedex_df['Pokémon'].isin(['CASTFORM SUNNY FORM','CASTFORM SNOWY FORM','CASTFORM RAINY FORM','DEOXYS NORMAL FORME', 'DEOXYS ATTACK FORME', 'DEOXYS DEFENSE FORME', 'DEOXYS SPEED FORME']), 'Hoenn Pokedex'] = True
pokedex_df.head()

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex,Hoenn Pokedex
0,1,BULBASAUR,Grass|Poison,318,45,49,49,65,65,45,1,True,True,NaN
1,2,IVYSAUR,Grass|Poison,405,60,62,63,80,80,60,1,True,True,NaN
2,3,VENUSAUR,Grass|Poison,525,80,82,83,100,100,80,1,True,True,NaN
3,4,CHARMANDER,Fire,309,39,52,43,60,50,65,1,True,True,NaN
4,5,CHARMELEON,Fire,405,58,64,58,80,65,80,1,True,True,NaN


### Pass Sinnoh Pokedex URL into Scraping Function and Prepare Dataframe for Join

In [25]:
# This dataset contains Pokemon that are registered in the Sinnoh regional pokedex (Pokemon Diamond/Pearl/Platinum), a combination of generations 1-4 Pokémon 
sinnoh_url='https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_Sinnoh_Pok%C3%A9dex_number'
scrape_table(sinnoh_url)
sinnoh_pokedex = df
# Apply transformations
sinnoh_pokedex.columns = sinnoh_pokedex.iloc[1]
sinnoh_pokedex = sinnoh_pokedex.iloc[2:]
sinnoh_pokedex = sinnoh_pokedex[['Pokémon']]
sinnoh_pokedex['Pokémon'] = sinnoh_pokedex['Pokémon'].str.upper()
sinnoh_pokedex = sinnoh_pokedex[sinnoh_pokedex['Pokémon'] != 'Pokémon']
sinnoh_pokedex = sinnoh_pokedex[sinnoh_pokedex.index <= 226]
sinnoh_pokedex['Sinnoh Pokedex'] = True
sinnoh_pokedex = sinnoh_pokedex.drop_duplicates()
sinnoh_pokedex.head()

1,Pokémon,Sinnoh Pokedex
2,TURTWIG,True
3,GROTLE,True
4,TORTERRA,True
5,CHIMCHAR,True
6,MONFERNO,True


#### **Many Sinnoh and Unova Pokemon have multiple forms and are thus named multiple times in the Sinnoh Pokedex (Burmy/Wormadam, Shellos/Gastrodon, Rotom, Giratina, Shaymin, Basculin, Kyurem, etc). However, these variants have the same base stats and Pokedex number and are thus being neglected when joined to the core Pokedex table. 

#### Merge Gen 4 Dataframe to Pokedex

In [28]:
pokedex_df=join(pokedex_df, sinnoh_pokedex)
pokedex_df.loc[pokedex_df['Pokémon'].isin(['BURMY PLANT CLOAK','BURMY SANDY CLOAK','BURMY TRASH CLOAK','WORMADAM PLANT CLOAK','WORMADAM SANDY CLOAK','WORMADAM TRASH CLOAK','ROTOM HEAT FORME','ROTOM WASH FORME','ROTOM FROST FORME','ROTOM FAN FORME','ROTOM MOW FORME','HEATRAN','REGIGIGAS','GIRATINA DISTORTION FORME','CRESSELIA','PHIONE','MANAPHY','DARKRAI','ARCEUS','SHAYMIN LAND FORME','SHAYMIN SKY FORME']), 'Sinnoh Pokedex'] = True
pokedex_df.head()

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex,Hoenn Pokedex,Sinnoh Pokedex
0,1,BULBASAUR,Grass|Poison,318,45,49,49,65,65,45,1,True,True,NaN,NaN
1,2,IVYSAUR,Grass|Poison,405,60,62,63,80,80,60,1,True,True,NaN,NaN
2,3,VENUSAUR,Grass|Poison,525,80,82,83,100,100,80,1,True,True,NaN,NaN
3,4,CHARMANDER,Fire,309,39,52,43,60,50,65,1,True,True,NaN,NaN
4,5,CHARMELEON,Fire,405,58,64,58,80,65,80,1,True,True,NaN,NaN


### Pass Unova Pokedex URL into Scraping Function and Prepare Dataframe for Join

In [30]:
# This dataset contains Pokemon that are registered in the Unova regional pokedex (Pokemon Black/White only), only generation 5 Pokemon
blackwhite_url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_Unova_Pok%C3%A9dex_number_in_Pok%C3%A9mon_Black_and_White'
scrape_table(blackwhite_url)
blackwhite_pokedex = df
# Apply transformations
blackwhite_pokedex.columns = blackwhite_pokedex.iloc[1]
blackwhite_pokedex = blackwhite_pokedex.iloc[2:]
blackwhite_pokedex = blackwhite_pokedex[['Pokémon']]
blackwhite_pokedex['Pokémon'] = blackwhite_pokedex['Pokémon'].str.upper()
blackwhite_pokedex = blackwhite_pokedex[blackwhite_pokedex['Pokémon'] != 'Pokémon']
blackwhite_pokedex = blackwhite_pokedex[blackwhite_pokedex.index <= 164]
blackwhite_pokedex['Unova Pokedex (Black/White)'] = True
blackwhite_pokedex = blackwhite_pokedex.drop_duplicates()
blackwhite_pokedex.head()

1,Pokémon,Unova Pokedex (Black/White)
2,VICTINI,True
3,SNIVY,True
4,SERVINE,True
5,SERPERIOR,True
6,TEPIG,True


#### Merge Gen 5 Dataframe to Pokedex

In [32]:
pokedex_df=join(pokedex_df, blackwhite_pokedex)
pokedex_df.loc[pokedex_df['Pokémon'].isin(['DARMANITAN STANDARD MODE','DARMANITAN ZEN MODE','TORNADUS THERIAN FORME','THUNDURUS THERIAN FORME','LANDORUS THERIAN FORME','KELDEO RESOLUTE FORM','MELOETTA PIROUETTE FORME','KYUREM WHITE','KYUREM BLACK']), 'Unova Pokedex (Black/White)'] = True
pokedex_df.head()

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex,Hoenn Pokedex,Sinnoh Pokedex,Unova Pokedex (Black/White)
0,1,BULBASAUR,Grass|Poison,318,45,49,49,65,65,45,1,True,True,NaN,NaN,NaN
1,2,IVYSAUR,Grass|Poison,405,60,62,63,80,80,60,1,True,True,NaN,NaN,NaN
2,3,VENUSAUR,Grass|Poison,525,80,82,83,100,100,80,1,True,True,NaN,NaN,NaN
3,4,CHARMANDER,Fire,309,39,52,43,60,50,65,1,True,True,NaN,NaN,NaN
4,5,CHARMELEON,Fire,405,58,64,58,80,65,80,1,True,True,NaN,NaN,NaN


#### **The Black 2/White 2 games use a completely different Pokedex than Black/White 1, thus Black 2/White 2 will be treated as its own regional Pokedex.

### Pass Unova (Black/White 2) Pokedex URL into Scraping Function and Prepare Dataframe for Join

In [35]:
# This dataset contains Pokemon that are registered in the Unova regional pokedex (Pokemon Black 2/White 2), a combination of generations 1-5 pokemon
blackwhite2_url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_Unova_Pok%C3%A9dex_number_in_Pok%C3%A9mon_Black_2_and_White_2'
scrape_table(blackwhite2_url)
blackwhite2_pokedex = df
# Apply transformations
blackwhite2_pokedex.columns = blackwhite2_pokedex.iloc[1]
blackwhite2_pokedex = blackwhite2_pokedex.iloc[2:]
blackwhite2_pokedex = blackwhite2_pokedex[['Pokémon']]
blackwhite2_pokedex['Pokémon'] = blackwhite2_pokedex['Pokémon'].str.upper()
blackwhite2_pokedex = blackwhite2_pokedex[blackwhite2_pokedex['Pokémon'] != 'Pokémon']
blackwhite2_pokedex = blackwhite2_pokedex[blackwhite2_pokedex.index <= 318]
blackwhite2_pokedex['Unova Pokedex (Black2/White2)'] = True
blackwhite2_pokedex = blackwhite2_pokedex.drop_duplicates()
blackwhite2_pokedex.head()

1,Pokémon,Unova Pokedex (Black2/White2)
2,VICTINI,True
3,SNIVY,True
4,SERVINE,True
5,SERPERIOR,True
6,TEPIG,True


#### Merge Gen 5 (Black/White 2) Dataframe to Pokedex

In [37]:
pokedex_df=join(pokedex_df,blackwhite2_pokedex)
pokedex_df.loc[pokedex_df['Pokémon'].isin(['CASTFORM SUNNY FORM','CASTFORM RAINY FORM','CASTFORM SNOWY FORM','DARMANITAN STANDARD MODE','DARMANITAN ZEN MODE','TORNADUS THERIAN FORME','THUNDURUS THERIAN FORME','LANDORUS THERIAN FORME','KELDEO RESOLUTE FORM','MELOETTA PIROUETTE FORME','KYUREM WHITE','KYUREM BLACK']), 'Unova Pokedex (Black2/White2)'] = True
pokedex_df.head()

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex,Hoenn Pokedex,Sinnoh Pokedex,Unova Pokedex (Black/White),Unova Pokedex (Black2/White2)
0,1,BULBASAUR,Grass|Poison,318,45,49,49,65,65,45,1,True,True,NaN,NaN,NaN,NaN
1,2,IVYSAUR,Grass|Poison,405,60,62,63,80,80,60,1,True,True,NaN,NaN,NaN,NaN
2,3,VENUSAUR,Grass|Poison,525,80,82,83,100,100,80,1,True,True,NaN,NaN,NaN,NaN
3,4,CHARMANDER,Fire,309,39,52,43,60,50,65,1,True,True,NaN,NaN,NaN,NaN
4,5,CHARMELEON,Fire,405,58,64,58,80,65,80,1,True,True,NaN,NaN,NaN,NaN


### Fill Missing Regional Pokedex Values as False (left join only appends True to the dataframe)

##### Each regional pokedex column will specify the Pokemon that exist in that region's pokedex, as is in the video games.

##### Regional pokedexes contain Pokemon from multiple different generations, which is why these columns were created and added to the national pokedex dataframe.

In [41]:
regional_pokedexes = ['Kanto Pokedex','Johto Pokedex','Hoenn Pokedex','Sinnoh Pokedex','Unova Pokedex (Black/White)','Unova Pokedex (Black2/White2)']
for dex in regional_pokedexes:
    pokedex_df[dex] = pokedex_df[dex].fillna(False).infer_objects(copy = False)
pokedex_df = pokedex_df.drop_duplicates()

### Final National Pokedex Dataframe

In [43]:
pokedex_df

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex,Hoenn Pokedex,Sinnoh Pokedex,Unova Pokedex (Black/White),Unova Pokedex (Black2/White2)
0,1,BULBASAUR,Grass|Poison,318,45,49,49,65,65,45,1,True,True,False,False,False,False
1,2,IVYSAUR,Grass|Poison,405,60,62,63,80,80,60,1,True,True,False,False,False,False
2,3,VENUSAUR,Grass|Poison,525,80,82,83,100,100,80,1,True,True,False,False,False,False
3,4,CHARMANDER,Fire,309,39,52,43,60,50,65,1,True,True,False,False,False,False
4,5,CHARMELEON,Fire,405,58,64,58,80,65,80,1,True,True,False,False,False,False
5,6,CHARIZARD,Fire|Flying,534,78,84,78,109,85,100,1,True,True,False,False,False,False
6,7,SQUIRTLE,Water,314,44,48,65,50,64,43,1,True,True,False,False,False,False
7,8,WARTORTLE,Water,405,59,63,80,65,80,58,1,True,True,False,False,False,False
8,9,BLASTOISE,Water,530,79,83,100,85,105,78,1,True,True,False,False,False,False
9,10,CATERPIE,Bug,195,45,30,35,20,20,45,1,True,True,False,False,False,False


### Filter Pokedex

In [75]:
# More user functionality explored in 'Widgets' notebook
pokedex_df[pokedex_df['Pokémon']== 'ARCEUS']

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex,Hoenn Pokedex,Sinnoh Pokedex,Unova Pokedex (Black/White),Unova Pokedex (Black2/White2)
507,493,ARCEUS,Normal,720,120,120,120,120,120,120,4,False,False,False,True,False,False


In [48]:
pokedex_df[pokedex_df['Johto Pokedex'] == True].sort_values('Type')

,#,Pokémon,Type,Base Stats,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Kanto Pokedex,Johto Pokedex,Hoenn Pokedex,Sinnoh Pokedex,Unova Pokedex (Black/White),Unova Pokedex (Black2/White2)
126,127,PINSIR,Bug,500,65,125,100,55,70,85,1,True,True,True,False,False,True
10,11,METAPOD,Bug,205,50,20,55,25,25,30,1,True,True,False,False,False,False
9,10,CATERPIE,Bug,195,45,30,35,20,20,45,1,True,True,False,False,False,False
202,204,PINECO,Bug,290,50,65,90,35,35,15,2,False,True,False,False,False,False
212,214,HERACROSS,Bug|Fighting,500,80,125,75,40,95,85,2,False,True,True,True,False,True
11,12,BUTTERFREE,Bug|Flying,395,60,45,50,90,80,70,1,True,True,False,False,False,False
122,123,SCYTHER,Bug|Flying,500,70,110,80,55,80,105,1,True,True,False,True,False,False
164,166,LEDIAN,Bug|Flying,390,55,35,50,55,110,85,2,False,True,False,False,False,False
163,165,LEDYBA,Bug|Flying,265,40,20,30,40,80,55,2,False,True,False,False,False,False
191,193,YANMA,Bug|Flying,390,65,65,45,75,45,95,2,False,True,False,True,False,True
